In [1]:
import pandas as pd
import numpy as np
import os
from pandas.tseries.offsets import BDay
from hsmm_core.utils import mc_limiting_distribution, states_from_limit_dist
from hsmm_core.observation_models import ExpIndMixDiracGauss
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hsmm_runner import HmmCalibration

from hsmm_core.hmm import hmm_impl

from hsmm_core.data_utils import DataLoader, TradingHours
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import pickle
from hsmm_core.consts import InitialisationMethod
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
sc = StandardScaler()


In [28]:
def listdirs(folder): #return only directories from a master folder
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]

def find_path(basepath, stringname):
    for fname in os.listdir(basepath):
        path = os.path.join(basepath, fname)
        if os.path.isdir(path):
            if stringname in fname:
                return path
 


In [29]:

data_dir = os.getenv('FINANCE_DATA') #main directory
# features_path = os.path.join(os.path.expanduser("~"), 'Data/features_models/features/') #where features are saved
# labels_path = os.path.join(os.path.expanduser("~"), 'Data/features_models/labels') #where labels are saved
# ticker_labels_path = os.path.join(labels_path, ticker + '/NON_DIRECTIONAL')

In [34]:
listdirs(data_dir)
rx_path = find_path(data_dir, 'RX1')
rx_files = os.listdir(find_path(data_dir, 'RX1'))
file_idx= 12
file_location = os.path.join(rx_path, rx_files[file_idx])

In [37]:
all_data_file =pd.read_csv(file_location,index_col=0)
trades = all_data_file[all_data_file['type']=='TRADE']

In [51]:
import datetime as dt
from dateutil.parser import parse



In [58]:
test=trades['time'][0]
datetime_test=parse(test)
x=datetime_test.time()

In [59]:
x

datetime.time(6, 1, 7, 515000)

In [61]:
x_tdelta = dt.timedelta(hours=x.hour, minutes=x.minute, seconds=x.second, microseconds=x.microsecond)

In [70]:
x_tdelta.total_seconds()

21667.515

In [73]:

trades.copy()

,size,time,type,value
0,401,2018-07-30 06:01:07.515,TRADE,162.13
3,1,2018-07-30 06:01:07.515,TRADE,162.13
8,7,2018-07-30 06:01:07.527,TRADE,162.13
10,12,2018-07-30 06:01:07.527,TRADE,162.13
18,3,2018-07-30 06:01:07.527,TRADE,162.14
19,4,2018-07-30 06:01:07.527,TRADE,162.15
23,2,2018-07-30 06:01:07.527,TRADE,162.14
26,1,2018-07-30 06:01:07.527,TRADE,162.14
28,1,2018-07-30 06:01:07.529,TRADE,162.13
37,1,2018-07-30 06:01:07.557,TRADE,162.14
